In [23]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import scipy

movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
movie_texts = movies['overview'].apply(str).to_list()
movie_titles = movies['title'].apply(str).to_list()
text_tokens = []
for i in range(len(movie_texts)):
    if movie_titles[i] != 'nan' and movie_texts[i] != 'nan':
        text = movie_titles[i] + " " + movie_texts[i]
        text_tokens.append(gensim.utils.simple_preprocess(text))

tagged_text = [TaggedDocument(t, [i]) for i, t in enumerate(text_tokens)]

In [22]:
#movies.astype({'id': 'int64'}).dtypes
movie_id_list = movies['id'].to_list()

for i in movie_id_list:
    try: 
        i = int(i)
    except ValueError:
        print(i)
        movies[movies.id != i]

1997-08-20
2012-09-29
2014-01-01


In [25]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=500, min_count=2, epochs=10)

model.build_vocab(tagged_text)

In [26]:
model.train(tagged_text, total_examples=model.corpus_count, epochs=model.epochs)

In [49]:
model.corpus_total_words

2494618

In [27]:
vecs = model.dv

In [51]:
print(vecs[1])

[-6.68990687e-02  6.34198934e-02  3.33163217e-02  4.57815304e-02
  8.38684756e-03 -3.48153786e-04  5.59236482e-02  5.33018708e-02
 -1.55969141e-02 -3.72254439e-02  2.49634124e-02 -3.78901660e-02
 -1.60023551e-02 -2.78191287e-02 -7.09323352e-03  2.07181759e-02
 -1.52211851e-02  1.25814800e-03 -8.63052346e-03  1.59456596e-01
 -1.88196748e-02 -3.27748135e-02  8.94453898e-02 -6.93789348e-02
 -1.25641711e-02  4.26690169e-02  1.43227857e-02  9.86306090e-03
 -4.25450541e-02  3.70595418e-02  6.87369984e-03  5.10605164e-02
 -2.67784428e-02 -2.63541397e-02 -8.14803783e-03 -4.56207758e-03
 -6.65566176e-02 -4.64985631e-02 -1.56390145e-02 -2.95166038e-02
 -3.66968510e-04  3.04952525e-02  7.05199167e-02 -3.72771639e-04
 -6.43827673e-03  4.53803465e-02 -3.32962698e-03  1.76628735e-02
 -3.47177275e-02 -3.88665944e-02 -1.09650828e-02 -3.76111418e-02
  2.49760654e-02 -8.57616886e-02 -4.02323045e-02 -6.13580830e-03
  9.20314789e-02  3.32038775e-02  7.17262849e-02  3.66701297e-02
 -4.58420394e-03  5.31957

In [28]:
model.wv.most_similar('good')

[('bad', 0.5717833638191223),
 ('much', 0.5451714992523193),
 ('lot', 0.5375788807868958),
 ('nice', 0.5225197076797485),
 ('really', 0.5202868580818176),
 ('bit', 0.5062395930290222),
 ('simple', 0.5058395266532898),
 ('easy', 0.49777311086654663),
 ('always', 0.4823594093322754),
 ('close', 0.4812656044960022)]

In [29]:

ranks = []
second_ranks = []
for t in range(len(tagged_text)):
    inferred_vector = model.infer_vector(tagged_text[t].words)
    sims = model.dv.most_similar([inferred_vector], topn=100)
    try:
        rank = [docid for docid, sim in sims].index(t)
        ranks.append(rank)
    except ValueError:
        ranks.append(-1)
    second_ranks.append(sims[1])


'\nranks = []\nsecond_ranks = []\nfor t in range(len(tagged_text)):\n    inferred_vector = model.infer_vector(tagged_text[t].words)\n    sims = model.dv.most_similar([inferred_vector], topn=100)\n    try:\n        rank = [docid for docid, sim in sims].index(t)\n        ranks.append(rank)\n    except ValueError:\n        ranks.append(-1)\n    second_ranks.append(sims[1])\n'

In [30]:

import collections

counter = collections.Counter(ranks)
print(counter)
print(counter[0]/len(tagged_text))


'\nimport collections\n\ncounter = collections.Counter(ranks)\nprint(counter)\nprint(counter[0]/len(tagged_text))\n'

In [29]:
ratings_train, ratings_test = train_test_split(ratings_small, test_size=0.3)
print(len(ratings_train))
print(len(ratings_test))
ratings_train_merge = ratings_train.merge(links_small, on='movieId')
ratings_test_merge = ratings_test.merge(links_small, on='movieId')
print(len(ratings_train_merge))
print(len(ratings_test_merge))
movies['id'].convert_dtypes(str)
ratings_train_merge['tmdbId'].convert_dtypes(str)
#ratings_train_full_merge = ratings_train_merge.merge(movies, left_on='tmdbId', right_on='id')

70002
30002
70002
30002


0           107
1           107
2           107
3           107
4           107
          ...  
69997     42618
69998     42852
69999     13241
70000    208869
70001     13763
Name: tmdbId, Length: 70002, dtype: Int64

In [30]:
# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
ram = 12    # in GB
count_to_big = 0
count_false_int = 0

userid_list = ratings_train['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))

# get all movie vector rated from user
def get_user_rated_movies(ids):
    
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                movie_vector = vecs[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                #print("ValueError")
                pass


        #movie_vectors = scipy.sparse.vstack(movie_vector_list)
        movie_vectors = movie_vector_list
        return movie_vectors


In [31]:
userid_list = ratings_train['userId'].unique()
all_user_profiles = {}
for uid in userid_list:
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    user_rated_movies_vector_array = np.array(user_rated_movies_vector_list)
    user_vector = user_rated_movies_vector_array.sum(axis=0)
    #user_vector = np.mean(user_rated_movies_vector_array, axis=0)
    all_user_profiles[uid] = user_vector

In [32]:
recommendation_users = {}
for uid in userid_list:
    if uid in all_user_profiles.keys():
        sims = model.dv.most_similar([all_user_profiles[uid]], topn=len(model.dv))
        recommendation_users[uid] = sims
len(recommendation_users)

671

In [33]:
recommendation_users[1][:10]

[(33744, 0.9387999176979065),
 (39381, 0.9358327388763428),
 (43444, 0.9355639815330505),
 (38492, 0.9347152709960938),
 (43927, 0.9342442750930786),
 (41437, 0.9341621398925781),
 (36542, 0.9317388534545898),
 (44473, 0.9283714294433594),
 (23961, 0.9280211329460144),
 (32052, 0.9269023537635803)]

In [57]:
%%time

percentage_already_seen = []
for uid in userid_list:
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_tmdb_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            user_tmdb_list.append(int(tmdbId))
        except ValueError:
            print("ValueError: " + str(tmdbId))
    
    m_count = 0
    for mid, cos in user_recommendation[:len(user_profile)]:
        try:
            if int(mid) in user_tmdb_list:
                m_count += 1
        except ValueError:
            print("ValueError: " + str(mid) + " not in tmdbid list")    

    if m_count == len(user_profile):
        print(str(uid) + " : the first movies are already viewed")
    else:
        print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

    p = m_count/len(user_profile)
    percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))

0 of 85 in first recommended already seen
0 of 163 in first recommended already seen
0 of 210 in first recommended already seen
0 of 49 in first recommended already seen
12 of 1181 in first recommended already seen
ValueError: 607   NaN
Name: tmdbId, dtype: float64
11 of 698 in first recommended already seen
ValueError: 7588   NaN
Name: tmdbId, dtype: float64
9 of 746 in first recommended already seen
0 of 46 in first recommended already seen
0 of 33 in first recommended already seen
0 of 37 in first recommended already seen
ValueError: 607   NaN
Name: tmdbId, dtype: float64
1 of 260 in first recommended already seen
0 of 188 in first recommended already seen
0 of 166 in first recommended already seen
1 of 377 in first recommended already seen
0 of 236 in first recommended already seen
2 of 197 in first recommended already seen
3 of 428 in first recommended already seen
4 of 621 in first recommended already seen
0 of 20 in first recommended already seen
0 of 68 in first recommended alr

In [172]:
# vector size = 50
Average Percentage of already seen movies: 0.0024270959347025794
# vector size = 100, epochs = 10
Average Percentage of already seen movies: 0.0025281050916012454
# vector size = 100, epochs = 20
Average Percentage of already seen movies: 0.0016255834501928675
# vector size = 100, epochs = 20, count_min = 1
Average Percentage of already seen movies: 0.0014803785890319565
# vector_size=50, min_count=5, epochs=20
Average Percentage of already seen movies: 0.0015261212643458547
# vector_size=500, min_count=2, epochs=10, 
Average Percentage of already seen movies: 0.0018633842313585812

SyntaxError: invalid syntax (<ipython-input-172-61ab19dec88f>, line 2)

In [58]:
# Remove movies used to create User Vector

percentage_already_seen = []
cleaned_recommendations = {}
for uid in userid_list:
    if uid in all_user_profiles.keys():
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
        user_recommendation = recommendation_users[uid]
        user_profile_mid_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                count_false_int = count_false_int + 1
            
        m_count = 0
        for mid, cos in user_recommendation:
            try:
                if int(mid) in user_tmdb_list:
                    m_count += 1
                    user_recommendation.remove((mid,cos))
            except ValueError:
                print("ValueError: " + str(mid) + " not in tmdbid list")   

        cleaned_recommendations[uid] = user_recommendation

In [59]:
# Calculate Percentage of user rated movies from test split in recommended, top 10/50/100
top_value = 100
user_percentages = []
nan_count = 0
for uid in userid_list:
    user_profile = ratings_test.loc[ratings_test['userId'] == uid]
    found_movies = 0
    if uid in recommendation_users.keys():
        user_recommendations = cleaned_recommendations[uid]
        #print(len(user_recommendations))
        #tmdbId_list = user_profile['tmdbId'].to_list():
        user_profile_movie_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_movie_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                count_false_int = count_false_int + 1

        for mid, cos in user_recommendations[:top_value]:
            try:
                if int(mid) in user_tmdb_list:
                    found_movies = found_movies + 1
            except ValueError:
                print("ValueError: " + str(mid) + " not in tmdbid list")

       
    user_percentages.append(found_movies/len(user_profile))

mean_percentage = sum(user_percentages)/len(user_percentages)
print("Mean Percentage of good Recommendations (based on Test Split): " + str(mean_percentage))

Mean Percentage of good Recommendations (based on Test Split): 0.0005327239415896569


In [ ]:
# vector_size=500, min_count=2, epochs=10, 
Mean Percentage of good Recommendations (based on Test Split): 0.0010615522910604877
# vector_size=50, min_count=2, epochs=10, 
Mean Percentage of good Recommendations (based on Test Split): 0.0025566209200994917
# vector_size=500, min_count=2, epochs=10, 
Mean Percentage of good Recommendations (based on Test Split): 0.0005327239415896569

In [60]:
from sklearn import svm




In [81]:
import collections
test_reco = recommendation_users
for k,l in test_reco.items():
    l.sort(key=lambda x:x[0])
    test_reco[k] = l


In [97]:
u_reco = test_reco[1]
u_reco_list = list(map(list, zip(*u_reco)))
u_reco_cos_array = np.array(u_reco_list[1])

In [14]:
userid_list = ratings_train['userId'].unique()

movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))

def get_user_rated_movies_bitarray(ids):
    
        bitarray = np.zeros(len(movieid_list))
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                bitarray[index] = 1
            except ValueError:
                #print("ValueError")
                pass

        return bitarray


all_user_bitarrays = {}

for uid in userid_list[:1]:
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_rated_movies_bitarray = get_user_rated_movies_bitarray(user_profile['movieId'].to_list())
    all_user_bitarrays[uid] = user_rated_movies_bitarray

NameError: name 'ratings_train' is not defined